In [1]:
import pandas as pd
df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [2]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [3]:
df.Sex.replace({
    'M':1,
    'F':0
},inplace=True)

df.ExerciseAngina.replace({
    'N':0,
    'Y':1
},inplace=True)

df.ST_Slope.replace({
    'Up':1,
    'Down':2,
    'Flat':3
},inplace =True)



In [4]:
df = pd.get_dummies(df,drop_first=True).astype(int)

In [5]:
df.head()

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST
0,40,1,140,289,0,172,0,0,1,0,1,0,0,1,0
1,49,0,160,180,0,156,0,1,3,1,0,1,0,1,0
2,37,1,130,283,0,98,0,0,1,0,1,0,0,0,1
3,48,0,138,214,0,108,1,1,3,1,0,0,0,1,0
4,54,1,150,195,0,122,0,0,1,0,0,1,0,1,0


In [6]:
x = df.drop('HeartDisease',axis=1)
y = df.HeartDisease

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
x_scaled

array([[-1.4331398 ,  0.51595242,  0.41090889, ..., -0.22967867,
         0.81427482, -0.49044933],
       [-0.47848359, -1.93816322,  1.49175234, ..., -0.22967867,
         0.81427482, -0.49044933],
       [-1.75135854,  0.51595242, -0.12951283, ..., -0.22967867,
        -1.22808661,  2.03894663],
       ...,
       [ 0.37009972,  0.51595242, -0.12951283, ..., -0.22967867,
         0.81427482, -0.49044933],
       [ 0.37009972, -1.93816322, -0.12951283, ..., -0.22967867,
        -1.22808661, -0.49044933],
       [-1.64528563,  0.51595242,  0.30282455, ..., -0.22967867,
         0.81427482, -0.49044933]])

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,stratify=y,random_state=10)

In [9]:
x_train.shape

(688, 14)

In [10]:
x_test.shape

(230, 14)

In [11]:
230/688

0.33430232558139533

In [12]:
y_train.value_counts()

HeartDisease
1    381
0    307
Name: count, dtype: int64

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

scores = cross_val_score(SVC(),x,y,cv=5)
print(scores)

[0.61413043 0.78804348 0.69565217 0.71584699 0.61748634]


In [16]:
scores.mean()

0.686231884057971

In [20]:
from sklearn.ensemble import BaggingClassifier
bagged_model = BaggingClassifier(
    estimator = SVC(),
    n_estimators =100,
    max_samples =0.8,
    oob_score =True,
    random_state =0,
)

bagged_model.fit(x_train,y_train)
bagged_model.oob_score_

0.8590116279069767

In [26]:
bagged_model = BaggingClassifier(
    estimator =SVC(),
    n_estimators = 100,
    max_samples =0.8,
    oob_score = True,
    random_state = 0,
)
scores = cross_val_score(bagged_model,x,y,cv=5)
scores.mean()

0.6884057971014493

In [27]:
from sklearn.tree import DecisionTreeClassifier
scores = cross_val_score(DecisionTreeClassifier(),x,y,cv=5)
scores

array([0.73913043, 0.80434783, 0.77717391, 0.68306011, 0.67759563])

In [25]:
scores.mean()

0.7417379425041578

In [28]:
bagged = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators = 100,
    max_samples = 0.8,
    oob_score = True,
    random_state = 0

)
scores = cross_val_score(bagged,x,y,cv=5)
scores.mean()

0.8005880256593014